In [ ]:
import math
import time
import cv2
import numpy as np

class Tracker:
    def __init__(self, start_line=400, end_line=600, distance=50):
        self.center_points = {}  
        self.id_count = 0  
        self.entry_times = {}  
        self.start_line = start_line  
        self.end_line = end_line  
        self.distance = distance  
        self.speeds = {}  
        self.disappeared = {}  

    def update(self, objects_rect):
        objects_bbs_ids = []
        new_center_points = {}

        assigned_ids = set()  # Prevent duplicate IDs in the same frame

        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            if cy < self.start_line or cy > self.end_line:
                continue  

            closest_id = None
            min_dist = float('inf')

            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])
                if dist < 60 and dist < min_dist and id not in assigned_ids:  
                    closest_id = id
                    min_dist = dist

            if closest_id is not None:
                self.center_points[closest_id] = (cx, cy)
                new_center_points[closest_id] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, closest_id])
                self.disappeared[closest_id] = 0  
                assigned_ids.add(closest_id)

                # ✅ Debug: Print tracking details
                print(f"Vehicle {closest_id} updated: Position ({cx}, {cy})")

                # ✅ Debug: Entry Detection
                if self.start_line - 10 <= cy <= self.start_line + 10 and closest_id not in self.entry_times:
                    self.entry_times[closest_id] = time.time()
                    print(f"Vehicle {closest_id} ENTERED at {self.entry_times[closest_id]:.2f} sec")  

                # ✅ Debug: Exit Detection & Speed Calculation
                if self.end_line - 10 <= cy <= self.end_line + 10 and closest_id in self.entry_times:
                    exit_time = time.time()
                    time_taken = exit_time - self.entry_times[closest_id]  

                    if time_taken > 0:
                        speed = (self.distance / time_taken) * 3.6  
                        self.speeds[closest_id] = speed  
                        print(f" Vehicle {closest_id} EXITED at {exit_time:.2f} sec | Speed: {speed:.2f} km/h")  

                    del self.entry_times[closest_id]  

            else:
                self.center_points[self.id_count] = (cx, cy)
                new_center_points[self.id_count] = (cx, cy)
                self.disappeared[self.id_count] = 0  
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                assigned_ids.add(self.id_count)

                # ✅ Debug: New Vehicle Detection
                print(f"New Vehicle {self.id_count} detected at ({cx}, {cy})")

                self.id_count += 1

        # Remove lost objects
        for id in list(self.center_points.keys()):
            if id not in new_center_points:
                self.disappeared[id] += 1
                if self.disappeared[id] > 10:  
                    print(f"Vehicle {id} lost tracking")
                    self.center_points.pop(id)
                    self.disappeared.pop(id)
                    self.speeds.pop(id, None)  

        self.center_points = new_center_points.copy()
        return objects_bbs_ids

     
# Load YOLO model
net = cv2.dnn.readNet(r'C:\Users\Harin\visual studio\project\yolov4.weights', 
                      r'C:\Users\Harin\visual studio\project\yolov4.cfg')
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load COCO labels
with open(r'C:\Users\Harin\visual studio\project\coco.names', "r") as f:
    classes = f.read().strip().split("\n")

# Initialize tracker
tracker = Tracker(start_line=400, end_line=600, distance=50)

# Load video
cap = cv2.VideoCapture(r"C:\Users\Harin\OneDrive\Documents\traffic_crop.mp4")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Save Output Video
fourcc = cv2.VideoWriter_fourcc(*"XVID")  
out = cv2.VideoWriter(r"C:\Users\Harin\OneDrive\Documents\out_perfect.mp4", fourcc, 30, (frame_width, frame_height))  

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(output_layers)

    boxes, confidences, class_ids = [], [], []

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5 and class_id in [2, 3, 5, 7]:  
                center_x, center_y, w, h = (detection[:4] * [width, height, width, height]).astype("int")
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    filtered_boxes = [boxes[i] for i in indices.flatten()] if len(indices) > 0 else []

    tracked_objects = tracker.update(filtered_boxes)

    for obj in tracked_objects:
        x, y, w, h, object_id = obj
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {object_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Display Speed if available
        if object_id in tracker.speeds:
            speed_text = f"{tracker.speeds[object_id]:.2f} km/h"
            cv2.putText(frame, speed_text, (x, y - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Draw Start & End Lines
    cv2.line(frame, (0, tracker.start_line), (width, tracker.start_line), (0, 0, 255), 2)
    cv2.putText(frame, "Start Line", (10, tracker.start_line - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    cv2.line(frame, (0, tracker.end_line), (width, tracker.end_line), (255, 0, 0), 2)
    cv2.putText(frame, "End Line", (10, tracker.end_line - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    out.write(frame)  
    cv2.imshow("Speed Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()  
cv2.destroyAllWindows()

print("✅ Done! Check the output video and console for details.")


🆕 New Vehicle 0 detected at (835, 404)
🔵 Vehicle 0 updated: Position (830, 406)
🚗 Vehicle 0 ENTERED at 1741417086.93 sec
🔵 Vehicle 0 updated: Position (825, 410)
🔵 Vehicle 0 updated: Position (825, 410)
🔵 Vehicle 0 updated: Position (820, 411)
🔵 Vehicle 0 updated: Position (813, 417)
🔵 Vehicle 0 updated: Position (803, 418)
🔵 Vehicle 0 updated: Position (797, 421)
🔵 Vehicle 0 updated: Position (793, 426)
🔵 Vehicle 0 updated: Position (793, 426)
🔵 Vehicle 0 updated: Position (788, 428)
🔵 Vehicle 0 updated: Position (781, 432)
🔵 Vehicle 0 updated: Position (774, 436)
🔵 Vehicle 0 updated: Position (767, 440)
🔵 Vehicle 0 updated: Position (759, 444)
🔵 Vehicle 0 updated: Position (760, 444)
🔵 Vehicle 0 updated: Position (751, 446)
🔵 Vehicle 0 updated: Position (742, 450)
🔵 Vehicle 0 updated: Position (734, 455)
🔵 Vehicle 0 updated: Position (730, 458)
🔵 Vehicle 0 updated: Position (721, 462)
🔵 Vehicle 0 updated: Position (721, 462)
🔵 Vehicle 0 updated: Position (714, 469)
🔵 Vehicle 0 update